# Add Articles to DB
    1. Get the list of new articles from CSV file
    2. Get article data from crossref
    3. Add articles to DB

In [1]:
# Libraries
# library containign functions that read and write to csv files
import lib.handle_csv as csvh
# library for getting data from crossref
import lib.crossref_api as cr_api


input_file = "pop_searches/AddNewArticles202002.csv"

csv_articles, _ = csvh.get_csv_data(input_file,'Num')


cr_articles = {} # Num (from CHUK), DOI, type, and other fiedls from CR.
article_columns=["Num"]
# list of article authors from cross ref
cr_authors = {} # AuthorNum, Firstname, Middle name, Last Name
author_columns = ["AuthorNum", "FirstName", "MiddleName", "LastName"]
# list of article-author links
cr_article_authour_link = {} # AuthorNum, DOI
article_authour_columns = ["DOI","AuthorNum"]

for art_num in csv_articles:
    article_title = csv_articles[art_num]['Title']
    doi_text = csv_articles[art_num]['DOI']
    article_data = cr_api.getBibData(doi_text)
    print(doi_text, article_data)
    

10.1039/c9sc04905c {'indexed': {'date-parts': [[2020, 1, 3]], 'date-time': '2020-01-03T05:41:01Z', 'timestamp': 1578030061134}, 'reference-count': 46, 'publisher': 'Royal Society of Chemistry (RSC)', 'issue': '2', 'license': [{'URL': 'http://creativecommons.org/licenses/by/3.0/', 'start': {'date-parts': [[2019, 11, 18]], 'date-time': '2019-11-18T00:00:00Z', 'timestamp': 1574035200000}, 'delay-in-days': 0, 'content-version': 'vor'}], 'funder': [{'DOI': '10.13039/501100000266', 'name': 'Engineering and Physical Sciences Research Council', 'doi-asserted-by': 'publisher', 'award': ['EP/K014714/1', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014706/1', 'EP/I019693/1']}], 'content-domain': {'domain': ['rsc.org'], 'crossmark-restriction': True}, 'abstract': '<p>Protagonists in the standard SCR reaction have been caught in the act by modulation excitation IR &amp; XAFS spectroscopy.</p>', 'DOI': '10.1039/c9sc04905c', 'type': 'article-journal', 'created': {'date-parts': [[2019, 11, 18]], 'date-time': 

10.1002/cctc.201901955 {'indexed': {'date-parts': [[2020, 1, 14]], 'date-time': '2020-01-14T14:40:27Z', 'timestamp': 1579012827494}, 'reference-count': 80, 'publisher': 'Wiley', 'license': [{'URL': 'http://creativecommons.org/licenses/by/4.0/', 'start': {'date-parts': [[2020, 1, 14]], 'date-time': '2020-01-14T00:00:00Z', 'timestamp': 1578960000000}, 'delay-in-days': 0, 'content-version': 'vor'}, {'URL': 'http://doi.wiley.com/10.1002/tdm_license_1.1', 'start': {'date-parts': [[2020, 1, 14]], 'date-time': '2020-01-14T00:00:00Z', 'timestamp': 1578960000000}, 'delay-in-days': 0, 'content-version': 'tdm'}], 'funder': [{'DOI': '10.13039/100011889', 'name': 'Diamond Light Source', 'doi-asserted-by': 'publisher', 'award': []}, {'DOI': '10.13039/501100000740', 'name': 'University of St Andrews', 'doi-asserted-by': 'publisher', 'award': []}, {'DOI': '10.13039/501100000266', 'name': 'Engineering and Physical Sciences Research Council', 'doi-asserted-by': 'publisher', 'award': ['EP/K014714/1', 'EP

10.1039/c9dt03590g {'indexed': {'date-parts': [[2019, 12, 17]], 'date-time': '2019-12-17T10:40:51Z', 'timestamp': 1576579251781}, 'reference-count': 44, 'publisher': 'Royal Society of Chemistry (RSC)', 'issue': '1', 'license': [{'URL': 'http://creativecommons.org/licenses/by/3.0/', 'start': {'date-parts': [[2019, 12, 9]], 'date-time': '2019-12-09T00:00:00Z', 'timestamp': 1575849600000}, 'delay-in-days': 0, 'content-version': 'vor'}], 'funder': [{'DOI': '10.13039/501100000739', 'name': 'University of Southampton', 'doi-asserted-by': 'publisher', 'award': []}, {'DOI': '10.13039/501100000266', 'name': 'Engineering and Physical Sciences Research Council', 'doi-asserted-by': 'publisher', 'award': ['EP/K014714/1', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014706/1', 'EP/I019693/1']}], 'content-domain': {'domain': ['rsc.org'], 'crossmark-restriction': True}, 'abstract': '<p>This paper describes the steps involved in the mechanochemical synthesis of LaMnO<sub>3</sub> using X-ray absorption spectros

10.1039/c9cp02968k {}
10.1039/c9se01103j {}
